In [2]:
%load_ext autoreload
%autoreload 2

# Compute Performance Data and Save for Plotting P-Charts

In [4]:
%load_ext autoreload
%autoreload 2

import os
from performance import compute_and_save_performance
suffix = "_true.csv"
for entry in os.listdir("evaluation_data"):
    if suffix in entry:
        name = entry[:-len(suffix)]
        print(name)
        compute_and_save_performance(name)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
characteristic_bolete.h5
direct_bolete_lowres.h5
direct_bolete.h5
characteristic_bolete_lowres.h5
direct_ed_bolete_lowres.h5
direct_ed_bolete.h5


# Get baselines for p-charts
## Random Performance

In [1]:
%load_ext autoreload
%autoreload 2

from performance_driver import drive
drive(100)

Direct:
Species
Edibility
Characteristics:
Species
=====================================================================================================species done
Edibility
=====================================================================================================edibility done


## Perfect Species Performance

In [5]:
from performance import perfect_performance
print("Species")
perfect_performance("species")
print("Edibility")
perfect_performance("edibility")

Species
=====================================================================================================done
Edibility
=====================================================================================================done


True

## Compare tau hats

In [4]:
from performance import * 
from lookalikes import *
from data_utils import load_raw_eval_data
import os


def shared(tau, tau_hat):
    return len(np.intersect1d(tau, tau_hat))

suffix = "_true.csv"
entries = os.listdir("evaluation_data")

modeltypes = ["characteristic", "direct"]
filetypes = ["_bolete.h5", "_bolete_lowres.h5"]
ps = {filetype:{} for filetype in filetypes}
for p in range(100):
    p = p/100
    tau_hats = {}
    for modeltype in modeltypes:
        for filetype in filetypes:
            name = modeltype+filetype
            scores, y_pred, y_true, y_labels = load_raw_eval_data(name)
            tau_hats[name] = []
            for i in range(len(y_labels)):
                if modeltype == "direct":
                    i_scores = list(enumerate(scores[i]))
                    sorted_scores = sorted(i_scores, key=lambda x: x[1], reverse=True)
                    size_of_tau_hat = len(lookalikes(sorted_scores[0][0], p))
                    tau_hat = [idx for (idx, val) in sorted_scores[:size_of_tau_hat]]
                else:
                    tau_hat = species_from_feats(y_pred[i], p)
                tau_hats[name].append(tau_hat)
    
    for filetype in filetypes:
        tau_hat_char = tau_hats["characteristic"+filetype]
        tau_hat_dir = tau_hats["direct"+filetype]
        num_examples = len(tau_hat_char)

        percent_shared = 0
        print("=", end='')
        for i in range(num_examples):
            total_unique = set(list(tau_hat_char[i]) + list(tau_hat_dir[i]))
            total = len(total_unique)
            num_shared = shared(tau_hat_char[i], tau_hat_dir[i])
            percent_shared += num_shared/total
        percent_shared /= num_examples
        ps[filetype][p] = round(percent_shared,2)

for filetype in filetypes:
    p_vals = [val for val in ps[filetype].keys()]
    sims = [sim for sim in ps[filetype].values()]
    # print('p vals = ', p_vals)
    # print('sims = ', sims)
    np.savetxt("performance_data/" + filetype[1:] + "_tau_hat_similarity.csv", [p_vals, sims], delimiter=",", fmt="%f",)